In [1]:
import time
import os
from threading import Thread

class FileWatcher:
    def __init__(self, file_path, interval=5):
        """
        文件监控初始化
        :param file_path: 要监控的文件路径
        :param interval: 检查间隔（秒）
        """
        self.file_path = file_path
        self.interval = interval
        self.running = False
        self.last_content = None
        self.retry_count = 3  # 文件读取重试次数

    def _read_file(self):
        """带重试机制的文件读取"""
        for _ in range(self.retry_count):
            try:
                with open(self.file_path, 'r') as f:
                    return f.read().strip()
            except (IOError, PermissionError) as e:
                print(f"Read attempt failed: {str(e)}")
                time.sleep(0.5)
        return None

    def _monitor_loop(self):
        """监控主循环"""
        print(f"Starting monitoring on {self.file_path}")
        while self.running:
            try:
                # 检查文件存在性
                if os.path.exists(self.file_path):
                    # 读取文件内容
                    current_content = self._read_file()
                    
                    if current_content is not None:
                        # 检测内容变化
                        if current_content != self.last_content:
                            print(f"\n=== New data detected ==="
                                  f"\nTimestamp: {time.ctime()}"
                                  f"\nContent: {current_content[:50]}...")  # 显示前50字符
                            
                            # 更新记录
                            self.last_content = current_content
                            # 触发回调
                            self.on_data_update(current_content)
                else:
                    print(f"Waiting for file: {self.file_path}")

                time.sleep(self.interval)

            except Exception as e:
                print(f"Monitoring error: {str(e)}")
                time.sleep(1)

    def on_data_update(self, new_data):
        """数据更新回调（可被子类重写）"""
        # 在这里添加自定义处理逻辑
        pass

    def start(self):
        """启动监控"""
        if not self.running:
            self.running = True
            self.thread = Thread(target=self._monitor_loop, daemon=True)
            self.thread.start()
            print(f"FileWatcher started (Interval: {self.interval}s)")

    def stop(self):
        """停止监控"""
        self.running = False
        if self.thread.is_alive():
            self.thread.join()
        print("FileWatcher stopped")

# 使用示例
if __name__ == "__main__":
    print("=== PYNQ File Watcher Demo ===")
    
    # 初始化监控器
    watcher = FileWatcher(
        file_path="/home/xilinx/jupyter_notebooks/message.txt",
        interval=3  # 3秒检测一次
    )

    # 自定义处理逻辑（可选）
    class DataHandler(FileWatcher):
        def on_data_update(self, new_data):
            """示例数据处理：显示有效数值"""
            try:
                value = float(new_data)
                print(f"Processing sensor data: {value:.2f} ℃")
            except ValueError:
                print("Received invalid data")

    # 使用自定义处理器
    watcher = DataHandler("/home/xilinx/jupyter_notebooks/message.txt", 3)

    try:
        watcher.start()
        while True:
            time.sleep(1)  # 主线程保持运行
    except KeyboardInterrupt:
        watcher.stop()
        print("\nMonitoring terminated")

=== PYNQ File Watcher Demo ===
Starting monitoring on /home/xilinx/jupyter_notebooks/message.txt
FileWatcher started (Interval: 3s)
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for file: /home/xilinx/jupyter_notebooks/message.txt
Waiting for fi